# Uso de modelos de lenguaje

Importación de librerías.

In [1]:
import pickle, math, nltk
from typing import List, Dict, Tuple

Función para cargar los modelos generados, y para leer y tokenizar las oraciones de testing.

In [2]:
def load_ngram_model(filename: str) -> Dict[Tuple[str, ...], float]:
    """
    Carga un modelo de n-gramas desde un archivo binario guardado con pickle.
    
    Args:
        filename (str): Nombre del archivo desde el que se cargará el modelo.
        
    Returns:
        Dict[Tuple[str, ...], float]: Diccionario con las probabilidades de los n-gramas cargado desde el archivo.
    """
    with open(filename, 'rb') as file:
        ngram_probabilities = pickle.load(file)
    return ngram_probabilities

def read_and_tokenize(file_path: str) -> List[List[str]]:
    """
    Lee un archivo de texto, tokeniza las oraciones y retorna una lista de listas de tokens.
    
    Args:
        file_path: Ruta al archivo de texto.
        
    Returns:
        List[List[str]]: Lista de listas de tokens (cada oración tokenizada).
    """
    with open(file_path, 'r', encoding="utf-8") as f:
        text = f.read()

    sentences = text.strip().split('\n')
    tokenized_sentences = [sentence.split() for sentence in sentences]
    return tokenized_sentences

Cargamos modelos para ambos datasets, y leemos las oraciones de testeo.

In [3]:
unigram_probs_20N = load_ngram_model("20N_unigrams.pkl")
bigram_probs_20N = load_ngram_model("20N_bigrams.pkl")
trigram_probs_20N = load_ngram_model("20N_trigrams.pkl")

In [4]:
unigram_probs_BAC = load_ngram_model("BAC_unigrams.pkl")
bigram_probs_BAC = load_ngram_model("BAC_bigrams.pkl")
trigram_probs_BAC = load_ngram_model("BAC_trigrams.pkl")

In [7]:
test_sentences_20N = read_and_tokenize("20N_testing.txt")
test_sentences_BAC = read_and_tokenize("BAC_testing.txt")

## V. Cálculo de la perplejidad

La fórmula vista de perplejidad es la siguiente:

$$PP(S)=\left(\prod_{i=1}^T P\left(w_i \mid w_{i-n} \dots w_{i-1 }\right)\right)^{-1 / T}$$

Como las probabilidades manejadas en nuestros modelos de lenguaje son de orden muy pequeños, podemos usar las probabilidades en espacio logaritmico de la siguiente manera:

$$
\begin{aligned}
\log (P P(s)) & =\log \left(\left(\prod_{i=1}^T P\left(w_i \mid w_{i-n} \dots w_{i-1 }\right)\right)^{-1 / T}\right) \\
& =-\frac{1}{T} \log \left(\prod_{i=1}^T P\left(w_i \mid w_{i-n} \dots w_{i-1}\right)\right) \\
& = -\frac{1}{T} \sum_{i=1}^T \log \left( P\left(w_i \mid w_{i-n} \dots w_{i-1}\right) \right)
\end{aligned}
$$

Así,

$$PP(S)= \exp \left[  -\frac{1}{T} \sum_{i=1}^T \log \left( P\left(w_i \mid w_{i-n} \dots w_{i-1}\right) \right) \right]$$

La siguiente función implementa este razonamiento para un listado de oraciones de prueba.

In [9]:
def calculate_perplexity_log(model: Dict[Tuple[str, ...], float], test_sentences: List[List[str]], n: int) -> float:
    """
    Calcula la perplejidad de un modelo de lenguaje n-gramas, utilizando logaritmos para evitar overflow.

    Args:
        model (Dict[Tuple[str, ...], float]): modelo de n-gramas. Es un diccionario que contiene las probabilidades de los n-gramas.
        test_sentences (List[List[str]]): Lista de oraciones tokenizadas (listas de tokens) para probar el modelo.
        n (int): Tamaño del n-grama a utilizar.
    
    Returns:
        float: El valor de la perplejidad del modelo sobre las oraciones de prueba.
    """
    total_log_prob = 0
    total_ngrams = 0
    
    # Para cada oración en el conjunto de prueba
    for sentence in test_sentences:
        sentence_log_prob = 0
        ngram_count = 0
        
        # Genera los n-gramas de la oración actual
        n_grams = list(nltk.ngrams(sentence, n))

        for ngram in n_grams:
            if ngram in model:
                prob = model[ngram]
            else:
                prob = 1e-10  # Si el n-grama no está en el modelo, asignar una probabilidad baja. Así evitamos el logaritmo de 0 (indefinido).

            # Sumar el logaritmo de la probabilidad del n-grama
            sentence_log_prob += math.log(prob)
            ngram_count += 1
        
        # Sumar la probabilidad (en log) total de la oración a la probabilidad total
        total_log_prob += sentence_log_prob
        total_ngrams += ngram_count

    # Promedio de la probabilidad
    avg_log_prob = total_log_prob / total_ngrams
    
    # La perplejidad es el exponente negativo del promedio
    perplexity = math.exp(-avg_log_prob)
    
    return perplexity

In [12]:
print("20N Dataset results:\n")

perplexity_unigrams_20N = calculate_perplexity_log(unigram_probs_20N, test_sentences_20N, n=1)
print(f'Perplejidad del modelo de unigramas: {perplexity_unigrams_20N}')

perplexity_bigrams_20N = calculate_perplexity_log(bigram_probs_20N, test_sentences_20N, n=2)
print(f'Perplejidad del modelo de bigramas: {perplexity_bigrams_20N}')

perplexity_trigrams_20N = calculate_perplexity_log(trigram_probs_20N, test_sentences_20N, n=3)
print(f'Perplejidad del modelo de trigramas: {perplexity_trigrams_20N}')

20N Dataset results:

Perplejidad del modelo de unigramas: 963.8673012218808
Perplejidad del modelo de bigramas: 11517.645242993816
Perplejidad del modelo de trigramas: 1499439.081950886


In [13]:
print("BAC Dataset results:\n")

perplexity_unigrams_BAC = calculate_perplexity_log(unigram_probs_BAC, test_sentences_BAC, n=1)
print(f'Perplejidad del modelo de unigramas: {perplexity_unigrams_BAC}')

perplexity_bigrams_BAC = calculate_perplexity_log(bigram_probs_BAC, test_sentences_BAC, n=2)
print(f'Perplejidad del modelo de bigramas: {perplexity_bigrams_BAC}')

perplexity_trigrams_BAC = calculate_perplexity_log(trigram_probs_BAC, test_sentences_BAC, n=3)
print(f'Perplejidad del modelo de trigramas: {perplexity_trigrams_BAC}')

BAC Dataset results:

Perplejidad del modelo de unigramas: 764.772774714158
Perplejidad del modelo de bigramas: 1885.4522528755106
Perplejidad del modelo de trigramas: 217254.66092308247


## VI. Generación de texto

La siguiente función define la estrategia de generación de texto, dado un modelo de n-gramas, la oración inicial, la cantidad de gramas considerados, y la longitud máxima del texto a generar. Con excepción de unigramas, que simplemente agarra el token con mayor probabilidad, la estrategia de generación busca el contexto dado en el modelo para generar el siguiente token, extrayendo por simplicidad aquel con mayor probabilidad. La generación se detiene si:

- Se llega a `max_length`
- Se llega al token `"</s>"`
- No hay gramas posibles como continuación de la oración

In [5]:
def generate_text(model: Dict[Tuple[str, ...], float], start_sentence: str, n: int, max_length: int = 20) -> str:
    """
    Genera texto usando el modelo n-grama basado en el n-grama de mayor probabilidad.

    Args:
        model: Diccionario de n-gramas con sus probabilidades.
        start_sentence: Frase de inicio como string.
        max_length: Longitud máxima de la oración generada.
        n: Tamaño del n-grama (ej. 1 para unigramas).

    Returns:
        string: Oración generada.
    """

    sentence = start_sentence.split()

    if n == 1:
        # Para unigramas, seleccionamos el token con la mayor probabilidad en cada paso.
        for _ in range(max_length):
            next_token = max(model, key=model.get)
            if next_token == '</s>':
                break
            sentence.append(next_token[0])
    else:
        # Para n-gramas donde n > 1
        for _ in range(max_length):
            context_size = min(len(sentence), n-1)  # Extraer cuantos tokens de contexto tenemos
            context = tuple(sentence[-context_size:])  # Tomar el contexto disponible
            
            # Buscar las posibles continuaciones en el modelo
            possible_ngrams = [ngram for ngram in model if ngram[:context_size] == context]
            
            # Si no hay posibles continuaciones, detenemos la generación
            if not possible_ngrams:
                break
            
            # Seleccionar el n-grama con la mayor probabilidad
            next_token = max(possible_ngrams, key=lambda ngram: model[ngram])[-1]

            # Si llegamos al token de fin de oración, detener la generación
            if next_token == '</s>':
                break
            sentence.append(next_token)

            # Si hemos generado suficientes palabras, detener la generación
            if len(sentence) >= max_length:
                break
    
    return ' '.join(sentence)

In [10]:
generate_text(trigram_probs_BAC, "<s>", max_length= 50, n = 3)

'<s> s NUM NUM NUM NUM NUM NUM NUM NUM NUM NUM NUM NUM NUM NUM NUM NUM NUM NUM NUM NUM NUM NUM NUM NUM NUM NUM NUM NUM NUM NUM NUM NUM NUM NUM NUM NUM NUM NUM NUM NUM NUM NUM NUM NUM NUM NUM NUM NUM'

In [8]:
generate_text(trigram_probs_BAC, "<s> this", max_length= 50, n = 3)

'<s> this is the best of all the time'

In [9]:
generate_text(trigram_probs_BAC, "how do", max_length= 50, n = 3)

'how do you think you re not going to be a good thing'